# Tutorial Notebooks
This is the third in a series of how-to notebooks that walk through producing simulated slitless spectroscopy images using Grizli. This notebook describes a few things you may need to modify in your direct image fits files before giving them to Grizli for simulation work.

## Outline
1) Header
2) Image Rotation
3) Bonus: Disperion Trace and Cutout Size

In [1]:
# Let's work in the tutorial/fits_files folder to make our files easy to access
import os
os.chdir('/Users/keith/astr/research_astr/summer-roman-project/tutorial/fits_files') # Hardcoded  
# os.chdir(os.path.join(os.getcwd(), 'fits_files')) # Generalized

#### 1) Header
Grizli expects details in the header that may not be there in fake fits files. Specifically, it expects instrument and filter information. Addittionally, if you're using a grism config file that is not built-in to Grizli, you need to include that filepath in the header.

In [2]:
from astropy.io import fits

# Set filenames
original_file = "GRS_FOV0_roll0_dx0_dy0_SCA1_direct_final.fits"
modified_header_filename = "Modified_Header_GRS_FOV0_roll0_dx0_dy0_SCA1_direct_final.fits"

# Open fits
direct_fits = fits.open(original_file)

# Grizli looks for INSTRUME and FILTER in the Primary HDU
direct_fits[0].header["INSTRUME"] = "ROMAN"
direct_fits[0].header["FILTER"] = "det1" # There is no filter, but one is required. I've been putting the detector instead

# Grizli looks for CONFFILE filepath in the Image HDU
direct_fits[1].header["CONFFILE"] = "/Users/keith/astr/research_astr/FOV0/Roman.det1.07242020.conf"

# Write new fits file with modified headers
direct_fits.writeto(modified_header_filename, overwrite=True)

#### 2) Image Rotation
The Roman Space Telescope Grism disperses in the y direction. aXeSim disperses in the x direction. To correct for this, we rotate 270 degrees clockwise before dispersion and 90 degrees clockwise afte dispersion.

In [3]:
import numpy as np

# Set filenames
rotated_image_filename = "Rotated_GRS_FOV0_roll0_dx0_dy0_SCA1_direct_final.fits"

# Open fits
direct_fits = fits.open(modified_header_filename)

# Before dispersion rotation; k is the number of clockwise 90 degree rotations
direct_fits[1].data = np.rot90(direct_fits[1].data, k=3)

# Write new fits file with rotate image data
direct_fits.writeto(rotated_image_filename, overwrite=True)

#### 3) Bonus: Dispersion Trace and Cutout Size
When dispersing an object, Grizli takes a cutout around that object and disperses within that cutout. By default, the cutout size is determined by the size of the object alone. However, the Roman Grism's trace is sometimes larger than the cutout size. This will cause objects to fail to disperse. To prevent this, we can set a size manually. Smaller cutout sizes reduce compute times, so we want to choose the smallest neccessary cutout size. 

One way to find this size is using the Scipy Optimize module as shown below. We want the largest displacement within our bounds. So, we need to compare the absolute values of the min and max. Scipy can only find the min. So, we run Scipy twice: once with the dy function and once with a negated dy function. We compare those two values for maximum absolute value. The manually set cutout size must be an integer larger than that float value.

In [4]:
# Import and set config_file
from scipy.optimize import minimize, Bounds
from collections import OrderedDict
config_file = "Roman.det1.07242020.conf"

# Parse config file
conf = OrderedDict()
lines = open(config_file).readlines()

for line in lines:
    if (line.startswith('#')) | (line.strip() == '') | ('"' in line):
                continue

    if line.startswith("DYDX_A_"):
        spl = line.split(';')[0].split('#')[0].split()
        param = spl[0]
        value = np.cast[float](spl[1:])
        conf[param] = value

# Pull out the coefficients
b = {}
for n, dydx in enumerate(conf.keys()):
    for m, coef in enumerate(conf[dydx]):
        b[f"{n},{m}"] = coef

In [5]:
# Define dispersion trace functions

# Field Dependence 
a_0 = lambda i, j: b["0,0"] + b["0,1"]*i + b["0,2"]*j + b["0,3"]*i**2 + b["0,4"]*i*j + b["0,5"]*j**2
a_1 = lambda i, j: b["1,0"] + b["1,1"]*i + b["1,2"]*j + b["1,3"]*i**2 + b["1,4"]*i*j + b["1,5"]*j**2
a_2 = lambda i, j: b["2,0"] + b["2,1"]*i + b["2,2"]*j + b["2,3"]*i**2 + b["2,4"]*i*j + b["2,5"]*j**2

# The Negated Trace 
dy = lambda args: a_0(args[0],args[1]) + a_1(args[0],args[1])*args[2] + a_2(args[0],args[1])*args[2]**2
dy_negated = lambda args: -(a_0(args[0],args[1]) + a_1(args[0],args[1])*args[2] + a_2(args[0],args[1])*args[2]**2)

In [6]:
# Set bounds of the chip (0:4088, 0:4800), and dx (-800:800)
bounds = Bounds([0,0,-800],[4800,4800,800])

# Scipy Optimize
local_minimum = minimize(dy, x0=[4800,4800,800], bounds=bounds)
local_maximum_negated = minimize(dy_negated, x0=[4800,4800,800], bounds=bounds)

# Compare
max_displacement = max(abs(local_minimum.fun), abs(local_maximum_negated.fun))

print("Cutout Size must be greater than or equal to %i." %(int(max_displacement)+1))

Cutout Size must be greater than or equal to 77.
